In [1]:
import pandas as pd
from main import get_pages, clean_notion_data

In [2]:
# Extract
raw_pages = get_pages()
print(f"Extracted {len(raw_pages)} rows")

Extracted 1 rows


In [3]:
# Transform
df = clean_notion_data(raw_pages)
df.head() 

,page_id,habit_name,date,status,is_completed
0,2db5e100-a756-8061-a8ce-eb159a74243a,Test habit,2026-01-01,Done,True


In [7]:
def transform_to_star_schema(df):
    """
    Transforms flat Notion data into a Star Schema.
    """
    # 1. DIMENSION: Date
    # We create a reference table for every unique date found in your data
    dim_date = df[['date']].drop_duplicates().reset_index(drop=True)
    
    # Create the Date ID (e.g. 20260101)
    dim_date['date_id'] = dim_date['date'].dt.strftime('%Y%m%d').astype(int)
    
    # Extract useful attributes for analysis
    dim_date['year'] = dim_date['date'].dt.year
    dim_date['month'] = dim_date['date'].dt.month
    dim_date['day'] = dim_date['date'].dt.day
    dim_date['day_name'] = dim_date['date'].dt.day_name()
    dim_date['is_weekend'] = dim_date['date'].dt.weekday >= 5

    # 2. DIMENSION: Habit
    # We find every UNIQUE habit name (e.g., "Gym", "Read").
    dim_habit = df[['habit_name']].drop_duplicates().reset_index(drop=True)
    dim_habit['habit_id'] = dim_habit.index + 100 # Starting IDs at 100 looks cleaner

    # 3. FACT TABLE: Daily Logs
    # We merge the dimensions back to the original data to get the IDs.
    
    # Merge Date ID
    fact_table = df.merge(dim_date, on='date', how='left')
    
    # Merge Habit ID
    fact_table = fact_table.merge(dim_habit, on='habit_name', how='left')
    
    # Select only the IDs and Metrics for the final Fact Table
    fact_habits = fact_table[[
        'page_id',      # Unique ID for the row (Notion ID)
        'date_id',      # Foreign Key -> dim_date
        'habit_id',     # Foreign Key -> dim_habit
        'is_completed', # The Metric (Boolean)
        'status'        # The Metric (Text)
    ]]

    return {
        "dim_date": dim_date,
        "dim_habit": dim_habit,
        "fact_habits": fact_habits
    }

In [8]:
# Cell 5: Run the Transformation
data_model = transform_to_star_schema(df)

In [9]:
# Cell 6: Visual Inspection (The part you want!)
print("--- DATE DIMENSION ---")
display(data_model['dim_date'].head())

print("--- HABIT DIMENSION ---")
display(data_model['dim_habit'].head())

print("--- FACT TABLE ---")
display(data_model['fact_habits'].head())

--- DATE DIMENSION ---


,date,date_id,year,month,day,day_name,is_weekend
0,2026-01-01,20260101,2026,1,1,Thursday,False


--- HABIT DIMENSION ---


,habit_name,habit_id
0,Test habit,100


--- FACT TABLE ---


,page_id,date_id,habit_id,is_completed,status
0,2db5e100-a756-8061-a8ce-eb159a74243a,20260101,100,True,Done
